In [127]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from scipy.io import loadmat
from pathlib import Path
import matplotlib.patches as patches
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa
from scipy import stats
from scipy.stats import mannwhitneyu, ttest_rel
from scipy.signal import butter, filtfilt
import mne
import eeg_analysis.funcs4eeg as fe
import re
import imp
import ast
import behavior.func4behav as fb
imp.reload(fe)
imp.reload(fb)

from scipy import signal
from scipy.signal import resample
from scipy.ndimage import zoom

In [128]:
case_title = 'endo'
case_list = [case_title+' fast', case_title+' slow']
if 'endo' in case_title:
    tmax_list = [1.5+1+0.5+0.05+1, 1.5+1+1+0.05+1]
else:
    tmax_list = [1.5+0.033*4+0.5+0.05+1, 1.5+0.033*4+1+0.05+1]

fs = 1200
eeg_fast = np.empty((0, 32, int(tmax_list[0]*fs)+1))
eeg_slow = np.empty((0, 32, int(tmax_list[1]*fs)+1))
rt = np.empty((0,))
for i, case in enumerate(case_list):
    tmax = tmax_list[i]
    watch = '1 fixation'
    tmin = 0 # include fix or not?

    highpass = None # 0.3
    lowpass = None

    EP_lists, RT_lists = fe.pipeline_EP_RT(case, watch, tmin, tmax, hipass=highpass, lopass=lowpass, baseline=(0,0), move_baseline=False, detrend=1)
    eeg_tmp = np.vstack([EP_lists[0], EP_lists[2]]) # only for all before
    
    # make eeg to same length
    if 'slow' in case:
        eeg_slow = np.vstack((eeg_slow, eeg_tmp))
    else:
        eeg_fast = np.vstack((eeg_fast, eeg_tmp))

    rt_tmp = np.hstack([RT_lists[0], RT_lists[2]])
    rt = np.hstack((rt, rt_tmp))


In [129]:
freq_bands = [[4, 8], [8, 12], [12, 30], [30, 50]]
endo_fast_stages = [[0, 1.5], [1.5, 1.5+1], [1.5+1, 1.5+1+0.5], [1.5+1+0.5+0.05, 1.5+1+0.5+0.05+1], [0, 1.5+1+0.5+0.05+1]]
endo_slow_stages = [[0, 1.5], [1.5, 1.5+1], [1.5+1, 1.5+1+1], [1.5+1+1+0.05, 1.5+1+1+0.05+1], [0, 1.5+1+1+0.05+1]]
exo_fast_stages = [[0, 1.5], [1.5, 1.5+0.033*4], [1.5+0.033*4, 1.5+0.033*4+0.5], [1.5+0.033*4+0.5+0.05, 1.5+0.033*4+0.5+0.05+1], [0, 1.5+0.033*4+0.5+0.05+1]]
exo_slow_stages = [[0, 1.5], [1.5, 1.5+0.033*4], [1.5+0.033*4, 1.5+0.033*4+1], [1.5+0.033*4+1+0.05, 1.5+0.033*4+1+0.05+1], [0, 1.5+0.033*4+1+0.05+1]]

In [130]:
bp_table = np.zeros((eeg_fast.shape[0]+eeg_slow.shape[0], 32, 5, 4)) # trials, 32 channels, 5 stages, 4 frequency bands
eeg_list = [eeg_fast, eeg_slow]
if 'endo' in case_title:
    stages_list = [endo_fast_stages, endo_slow_stages]
else:
    stages_list = [exo_fast_stages, exo_slow_stages]

adjust = 0
for i, eeg in enumerate(eeg_list):
    stages = stages_list[i]
    # add stim channel
    stim = np.zeros((1, eeg.shape[2]))
    stim[0,1] = 1

    for trial in range(eeg.shape[0]):
        # print(trial+adjust)
        data = eeg[trial,:,:]
        for j, stage in enumerate(stages):
            stage_start = int(stage[0]*fs)
            stage_end = int(stage[1]*fs)
            psds, freqs = mne.time_frequency.psd_array_multitaper(data[:,stage_start:stage_end], fs, fmin=0, fmax=51, verbose=False)
            for k,band in enumerate(freq_bands):
                bp_table[trial+adjust, :, j, k] = fe.band_power(psds, freqs, band)
    adjust = eeg.shape[0]
# 35.6 sec